In [1]:
import os
from matplotlib import image
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt
from binarization.binarization_functions import Binarizator
from binarization.evaluation_measure import Measure
from skimage.util import img_as_ubyte
from skimage.util import img_as_float32

ModuleNotFoundError: No module named 'skimage'

<h1>Comparation </h1>
I am going to compare the implemented binarization method with the Otsu Binarization Method.

In [ ]:
#load images
path = "../Data/ue1"
photo_name = "F2s.png"
#get all the file in the directory
arr = os.listdir(path)
dir = [file for file in arr if os.path.isdir(os.path.join(path, file)) ]

img = {}
gt_img = {}

for i in range(0, len(dir)-1):
    img[dir[i]] = image.imread(os.path.join(path,dir[i], photo_name))
    #img[i] = cv2.imread(os.path.join(path,dir[i], photo_name), cv2.IMREAD_GRAYSCALE)
    gt_img[dir[i]] = image.imread(os.path.join(path,dir[i], f"{dir[i]}GT.png"))

#plot the images
plt.figure(figsize=(15, 15))
for j in range(len(dir)-1):
    ax = plt.subplot(3, 3, j + 1)
    plt.imshow(img[dir[j]], cmap="gray")
    plt.title(dir[j])
    plt.axis("off")

<h2>Implemented Binarization Method</h2>

In [ ]:
window = 3
path = "../object"
binarizators = {}
for i in img.keys():
    print(f"Image: {i}")
    if os.path.isfile(os.path.join(path, i)):
        #if the binarizator on the image already exist
        print(f"file {os.path.join(path, i)} present")
        with open(os.path.join(path, i), "rb") as binarizator_file:
            bin = pickle.load(binarizator_file)

    #the file does not exist
    else:
        print(f"file {os.path.join(path, i)} NOT present")
        bin = Binarizator(img[i], window, i)
        #do the binarization
        bin.contrast_image_constructor()
        bin.high_contrast_image()
        bin.pixels_classification()

        #load on file the binarizator
        with open(os.path.join(path, i), "wb") as file:
            pickle.dump(bin, file)
    binarizators[i] = bin

In [ ]:
#plot the images
plt.figure(figsize=(15, 15))
i = 0
for j in img.keys():
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(binarizators[j].get_final_image(), cmap="gray")
    plt.title(binarizators[j].getID())
    plt.axis("off")
    i+= 1

<h2>Otsu Tecnique</h2>

In [ ]:
output_otsu = {}
for i in img.keys():
    image = img_as_ubyte(img[i])
    ret, thresh = cv2.threshold(image, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU )
    output_otsu[i] = thresh
output_otsu["z592"]

In [ ]:
#plot the images
i = 0
plt.figure(figsize=(15, 15))
for j in output_otsu.keys():
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(output_otsu[j], cmap="gray")
    plt.title(j)
    plt.axis("off")
    i += 1

<h2>Compute the measure</h2>

I will compute the avarage value of the following measures for every image:
- F-measure
- pF-measure
- PSNR (Peak Signal-to-noise ratio)
- DRD (Distance-Reciprocal Distortion Measure)


In [ ]:
#f_measure
f_measure = 0
f_measure_otsu = 0

#p_f_measure
pf_measure = 0
pf_measure_otsu = 0

#PSNR
psnr = 0
psnr_otsu = 0

#DRD
drd = 0
drd_otsu = 0

In [ ]:
path = "../measures"
for i in binarizators.keys():
    print(f"image: {i}")
    if os.path.isfile(os.path.join(path, i)):
        #if the binarizator on the image already exist
        print(f"file {os.path.join(path, i)} present")
        with open(os.path.join(path, i), "rb") as file:
            m = pickle.load(file)

    #the file does not exist
    else:
        print(f"file {os.path.join(path, i)} NOT present")
        m = Measure(binarizators[i].get_final_image(), gt_img[i])
        m.f_measure()
        m.p_f_measure()
        m.psnr()
        m.drd()

        #load on file
        with open(os.path.join(path, i), "wb") as file:
            pickle.dump(m, file)

    f_measure += m.getFmeasure()
    pf_measure += m.getPFmeasure()
    psnr += m.getPSNR()
    drd += m.getDRD()

f_measure /= len(binarizators)
pf_measure /= len(binarizators)
psnr /= len(binarizators)
drd /= len(binarizators) 

In [ ]:
for i in output_otsu.keys():
    #otsu
    print(f"image: {i}")
    file_name = i + "_otsu"
    if os.path.isfile(os.path.join(path, file_name)):
        #if the binarizator on the image already exist
        print(f"file {os.path.join(path,file_name )} present")
        with open(os.path.join(path, file_name), "rb") as file:
            m_otsu = pickle.load(file)

    #the file does not exist
    else:
        print(f"file {os.path.join(path, file_name)} NOT present")
        m_otsu = Measure(output_otsu[i], gt_img[i])
        m_otsu.f_measure()
        m_otsu.p_f_measure()
        m_otsu.psnr()
        m_otsu.drd()

        #load on file
        with open(os.path.join(path, file_name), "wb") as file:
            pickle.dump(m_otsu, file)

    f_measure_otsu += m_otsu.getFmeasure()
    pf_measure_otsu += m_otsu.getPFmeasure()
    psnr_otsu += m_otsu.getPSNR()
    drd_otsu += m_otsu.getDRD()

f_measure_otsu /= len(output_otsu)
pf_measure_otsu /= len(output_otsu)
psnr_otsu /= len(output_otsu)
drd_otsu /= len(output_otsu)


In [ ]:
#print measures
print(f"f measure: {f_measure}")
print(f"Pf measure: {pf_measure}")
print(f"PSNR: {psnr}")
print(f"DRD: {drd} \n")

print(f"Otsu measure")
print(f"f measure: {f_measure_otsu}")
print(f"Pf measure: {pf_measure_otsu}")
print(f"PSNR: {psnr_otsu}")
print(f"DRD: {drd_otsu} \n")